## Imports

In [1]:
from __future__ import annotations
from typing import Optional, Any, Tuple, List
import re

import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## Reading Datafiles

In [2]:
def read_csv(filename: str) -> pd.DataFrame:
    return pd.read_csv(filename, sep=',', encoding='latin-1')

train_and_val_data = read_csv('data/ds4420_kaggle_train_data.csv')
test_data = read_csv('data/ds4420_kaggle_test_data.csv')

def split_label(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    return data.drop('Label', axis=1), data['Label']

X_train_and_val, y_train_and_val = split_label(train_and_val_data)
X_train, X_val, y_train, y_val = train_test_split(X_train_and_val,
                                                      y_train_and_val,
                                                      test_size=0.2,
                                                      random_state=1)

X_test = test_data

## Custom Feature Extractors

In [3]:
class ThresholdClassifier(BaseEstimator, TransformerMixin):
    
    model: BaseEstimator
    threshold: int

    def __init__(self, model: BaseEstimator, threshold: int=0.5) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.model = model
        self.threshold = threshold

    def fit(self, X: pd.Series, y: Optional[Any]=None) -> ThresholdClassifier:
        self.model.fit(X, y)
        return self
    
    def predict(self, X: pd.Series) -> pd.DataFrame:
        return self.model.predict_proba(X)[:,1] > self.threshold

class InteractFeaturesTransformer(FunctionTransformer):
    """
    Multiplies two features together to yield a new feature.
    """
    
    def __init__(self) -> None:
        FunctionTransformer.__init__(self, self.__interact_features, validate=True)

    def __interact_features(self, X: pd.DataFrame) -> pd.Series:
        return X[:, 0:1] * X[:, 1:2]

class LengthFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into its length.
    """
    
    def __init__(self) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
    
    def fit(self, X: pd.Series, y: Optional[Any]=None) -> LengthFeatureExtractor:
        return self
        
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(lambda text: len(text))
        return yelling_feature.values.reshape(-1, 1)

class CapsBinaryFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into a binary feature where 1 denotes that at least one word in the text
    is completely capitalized, which is assumed to be yelling.
    """
    
    def __init__(self) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        
    def __text_contains_caps_binary(self, text: str) -> int:
        return int(any(word.isupper() for word in text.split(' ')))
    
    def fit(self, X: pd.Series, y: Optional[Any]=None) -> CapsBinaryFeatureExtractor:
        return self
        
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(self.__text_contains_caps_binary)
        return yelling_feature.values.reshape(-1, 1)

class CapsCountFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into a numeric feature count of the number of words in the text that are
    completely capitalized, which is assumed to be yelling.
    """
    
    def __init__(self) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
    
    def __text_caps_count(self, text: str) -> int:
        return int(sum(1 for word in text.split(' ') if word.isupper()))
    
    def fit(self, X: pd.Series, y: Optional[Any]=None) -> CapsCountFeatureExtractor:
        return self
    
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(self.__text_caps_count)
        return yelling_feature.values.reshape(-1, 1)

class CharBinaryFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into binary feature where 1 indicates the text containst at least one of
    the given character.
    """

    char: str
    
    def __init__(self, char: str) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)

        if len(char) != 1:
            raise ValueError('Character must be a string of length 1.')
        
        self.char = char
    
    def fit(self, X: pd.Series, y: Optional[Any]=None) -> ExclamationBinaryFeatureExtractor:
        return self
    
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(lambda text: 0 + (self.char in text))
        return yelling_feature.values.reshape(-1, 1)

class CharCountFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into a numeric feature count of the number of exclamation marks, which is
    assumed to be yelling.
    """

    char: str
    
    def __init__(self, char: str) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
    
        if len(char) != 1:
            raise ValueError('Character must be a string of length 1.')
        
        self.char = char

    def fit(self, X: pd.Series, y: Optional[Any]=None) -> ExclamationCountFeatureExtractor:
        return self
    
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(lambda text: text.count(self.char))
        return yelling_feature.values.reshape(-1, 1)

class SwearingFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Converts a text feature into a binary feature where 1 denotes swearing occurs, which is usually
    seen in text as a series of 2 or more asterisks, e.g. ****.
    """
    
    def __init__(self) -> None:
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
    
    def fit(self, X: pd.Series, y: Optional[Any]=None) -> SwearingFeatureExtractor:
        return self
    
    def transform(self, X: pd.Series) -> pd.DataFrame:
        yelling_feature = X.apply(lambda text: 0 + ('**' in text))
        return yelling_feature.values.reshape(-1, 1)

In [4]:
def clean_text(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

## Reusable Funtionality to Evaluate a Model and Save Results

In [5]:
def evaluate_and_save_results(pipeline: Pipeline, model_num: int, train_on_selected_text: bool=False) -> None:

    X_train_prepared = X_train.copy()
    X_val_prepared = X_train.copy()
    
    if train_on_selected_text:
        X_train_prepared['Text'] = X_train['Selected_Text']
        X_val_prepared['Text'] = X_val['Selected_Text']
    
    pipeline.fit(X_train_prepared, y_train)
    
    y_val_pred = pipeline.predict(X_val_prepared)
    y_test_pred = pipeline.predict(X_test)
    
    print(classification_report(y_val, y_val_pred))
    
    test_data_exportable = test_data.copy()
    test_data_exportable['Label'] = y_test_pred
    test_data_exportable = test_data_exportable[['ID', 'Label']]
    test_data_exportable.to_csv(f'outputs/model{model_num}.csv', index=False)

## A "Switchboard" to Enable and Disable Running Each Model

In [6]:
models_to_run = {
    'model1': False,
    'model2': False,
    'model3': False,
    'model4': False,
    'model5': False,
    'model6': False,
    'model7': False,
    'model8': False,
    'model9': False,
    'model10': False,
    'model11': False,
    'model12': False,
    'model13': False,
    'model14': True,
    'model15': True,
}

## Model 1

In [7]:
if models_to_run['model1']:
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', TfidfVectorizer(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', LogisticRegression(random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 1)

## Model 2

In [8]:
if models_to_run['model2']:
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', TfidfVectorizer(stop_words='english'), 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=200, random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 2)

## Model 3

In [9]:
if models_to_run['model3']:
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', CountVectorizer(stop_words='english'), 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=200, random_state=1))
        ]
    )
    
    evaluate_and_save_results(pipeline, 3)

## Model 4

In [10]:
if models_to_run['model4']:
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', CountVectorizer(stop_words='english'), 'Text'),
            ('numeric', MinMaxScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', MultinomialNB()),
        ]
    )
    
    evaluate_and_save_results(pipeline, 4)

## Model 5

In [11]:
if models_to_run['model5']:
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', CountVectorizer(stop_words='english'), 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', SVC(kernel='poly')),
        ]
    )
    
    evaluate_and_save_results(pipeline, 5)

## Model 6

In [12]:
if models_to_run['model6']:        
    text_vectorizer = CountVectorizer(
        stop_words='english',
        # ngram_range=(1, 2)
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('yelling', YellingBinaryFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=200, random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 6)

## Model 7

In [13]:
if models_to_run['model7']:
        
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
            ('exclamation', ExclamationCountFeatureExtractor(), 'Text'),
            ('swearing', SwearingFeatureExtractor(), 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', LogisticRegression(random_state=1, max_iter=1000)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 7)

In [14]:
if models_to_run['model8']:
    text_vectorizer = CountVectorizer(
        stop_words='english',
        # ngram_range=(1, 2)
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=200, random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 8)

In [15]:
if models_to_run['model9']:
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=225, random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 9)

In [16]:
if models_to_run['model10']:
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestClassifier(n_estimators=225, random_state=1)),
        ]
    )

    param_grid = {
        'n_estimators': [220, 225, 230],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 2],
        'max_features': [None, 'sqrt', 'log2']
    }
    
    grid_search = GridSearchCV(RandomForestClassifier(random_state=1), param_grid, cv=3, scoring='f1')
    grid_search.fit(preprocessor.fit_transform(X_train_and_val), y_train_and_val)

    print(grid_search.best_params_)
    
    # evaluate_and_save_results(pipeline, 10)

In [17]:
if models_to_run['model11']:
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )

    voting_classifier = VotingClassifier(
        estimators=[
            ('logistic_regression', LogisticRegression(random_state=1)),
            ('random_forest', RandomForestClassifier(n_estimators=225, random_state=1)),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('voting', voting_classifier),
        ]
    )
    
    evaluate_and_save_results(pipeline, 11)

In [18]:
if models_to_run['model12']:    
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', ThresholdClassifier(RandomForestClassifier(n_estimators=225, random_state=1), 0.55)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 12)

In [19]:
if models_to_run['model13']:
    
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            ('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            ('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(n_estimators=221, random_state=1)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 13)

In [20]:
if models_to_run['model14']:
    
    text_vectorizer = CountVectorizer(
        stop_words='english',
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_vectorizer, 'Text'),
            #('numeric', StandardScaler(), ['User_Age', 'Time_of_Post', 'Population_Density']),
            #('capitalization', CapsCountFeatureExtractor(), 'Text'),
        ]
    )
    
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(max_iter=1000, random_state=2)),
        ]
    )
    
    evaluate_and_save_results(pipeline, 14, train_on_selected_text=True)

ValueError: np.nan is an invalid document, expected byte or unicode string.